In [1]:
import cv2
import os
import numpy as np
import itertools
from matplotlib import pyplot as plt

In [2]:
import tensorflow as tf
from tensorflow import keras

2023-11-20 22:40:32.936835: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-20 22:40:33.484576: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
tf.config.list_physical_devices()

2023-11-20 22:40:44.367533: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-11-20 22:40:44.367563: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: s4m-g531gt-ubuntu
2023-11-20 22:40:44.367569: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: s4m-g531gt-ubuntu


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

2023-11-20 22:40:44.367660: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 520.61.5
2023-11-20 22:40:44.367677: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 520.61.5
2023-11-20 22:40:44.367682: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 520.61.5


# 1. Dataset

In [ ]:
data_path = os.path.join('face_dataset')

In [ ]:
IMG_SIZE = 128
INPUT_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
BUFFER_SIZE = 123
BATCH_SIZE = 32

In [ ]:
def create_positive_pairs(dataset_path):
    positive_pairs = []
    for person_folder in os.listdir(dataset_path):
        images = os.listdir(os.path.join(dataset_path, person_folder))
        pairs = list(itertools.combinations(images, 2))
        positive_pairs.extend([(person_folder, pair[0], person_folder, pair[1]) for pair in pairs])
    return np.array(positive_pairs)

In [ ]:
def create_negative_pairs(dataset_path, positive_pairs, len_list):
    negative_pairs = []

    for i in range(len(positive_pairs)):
        person_1, img_1, _, _ = positive_pairs[i]

        person_2 = 'person_' + str(np.random.randint(0, 104))
        while person_2 == person_1:
            person_2 = 'person_' + str(np.random.randint(0, 104))

        img_2 = 'img_' + str(np.random.randint(0, len_list[int(person_2.split('_')[-1])]))
        negative_pairs.append((person_1, img_1, person_2, img_2))
    return np.array(negative_pairs)

In [ ]:
def load_and_preprocess_image(image_path):
    byte_img = tf.io.read_file(image_path)
    img = tf.io.decode_jpeg(byte_img)
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0
    return img

In [ ]:
def preprocess_and_create_example(person_1, img_1, person_2, img_2, dataset_path):
    image_1 = load_and_preprocess_image(os.path.join(dataset_path, person_1, img_1))
    image_2 = load_and_preprocess_image(os.path.join(dataset_path, person_2, img_2))
    
    label = int(person_1 == person_2)
    
    return ((image_1, image_2), label)

In [ ]:
def create_dataset(dataset_path):
    positive = create_positive_pairs(dataset_path)
    len_dataset = 2 * len(positive)
    
    len_list = []
    for i in range(104):
        len_list.append(len(os.listdir(os.path.join(dataset_path, 'person_' + str(i)))))

    def generator():
        negative = create_negative_pairs(dataset_path, positive, len_list)
        data_pairs = np.concatenate((positive, negative), axis=0)
        np.random.shuffle(data_pairs)
        
        for person_1, img_1, person_2, img_2 in data_pairs:
            yield preprocess_and_create_example(person_1, img_1, person_2, img_2, dataset_path)

    output_signature = (
        (tf.TensorSpec(shape=INPUT_SHAPE, dtype=tf.float32),
        tf.TensorSpec(shape=INPUT_SHAPE, dtype=tf.float32)),
        tf.TensorSpec(shape=(), dtype=tf.float32)
    )

    tf_dataset = tf.data.Dataset.from_generator(generator, output_signature=output_signature)
    return tf_dataset, len_dataset

In [ ]:
dataset, dataset_size = create_dataset(data_path)

In [ ]:
for x in dataset.take(1):
    print(x)

In [ ]:
train = dataset.take(int(0.9 * dataset_size))
dev = dataset.skip(int(0.9 * dataset_size)).take(int(0.05 * dataset_size))
test = dataset.skip(int(0.9 * dataset_size)).skip(int(0.05 * dataset_size))

In [ ]:
train = train.shuffle(BUFFER_SIZE)
dev = dev.shuffle(BUFFER_SIZE)

In [ ]:
train = train.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
dev = dev.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test = test.batch(BATCH_SIZE)

In [ ]:
for x, y in train.take(1):
    print(x)

In [ ]:
def make_mobile_model(input_shape, embeddingDim=64):
    mobile_net = keras.applications.MobileNetV3Small(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    mobile_net.trainable = False
    flatten = keras.layers.Flatten()(mobile_net.output)
    dense = keras.layers.Dense(512, activation='relu')(flatten)
    
    model = keras.models.Model(inputs=mobile_net.input, outputs=dense)
    return model

In [ ]:
def create_siamese_model(input_shape):
    input_1 = keras.layers.Input(shape=input_shape)
    input_2 = keras.layers.Input(shape=input_shape)

    siamese_network = make_mobile_model(input_shape)
    encoded_1 = siamese_network(input_1)
    encoded_2 = siamese_network(input_2)

    distance = tf.abs(tf.subtract(encoded_1, encoded_2))

    output = keras.layers.Dense(1, activation='sigmoid')(distance)

    siamese_model = keras.models.Model(inputs=[input_1, input_2], outputs=output)

    return siamese_model

In [ ]:
model = create_siamese_model(INPUT_SHAPE)
model.summary()

In [ ]:
METRICS = [
    keras.metrics.BinaryAccuracy(),
    keras.metrics.F1Score(),
    keras.metrics.Precision(),
    keras.metrics.Recall()
]

In [ ]:
model.compile(
    optimizer='adam',
    loss=keras.losses.BinaryCrossentropy(),
    metrics=METRICS
)

In [ ]:
hist = model.fit(
    train,
    epochs=10,
    verbose=1,
    validation_data=dev
)